In [1]:
import numpy as np

In [2]:
def random_initialization(D,rank):
    number_of_documents = D.shape[0]
    number_of_terms = D.shape[1]
    U = np.random.uniform(1,2,(number_of_documents,rank))
    V = np.random.uniform(1,2,(rank,number_of_terms))
    return U,V.T

# NNDSVD method, Boutsidis & Gallopoulos, 2007
def nnd_svd_initialization(D,rank):
    u,s,v=np.linalg.svd(D,full_matrices=False)
    v=v.T
    w=np.zeros((D.shape[0],rank))
    h=np.zeros((rank,D.shape[1]))

    w[:,0]=np.sqrt(s[0])*np.abs(u[:,0])
    h[0,:]=np.sqrt(s[0])*np.abs(v[:,0].T)

    for i in range(1,rank):
        
        ui=u[:,i]
        vi=v[:,i]
        ui_pos=(ui>=0)*ui
        ui_neg=(ui<0)*-ui
        vi_pos=(vi>=0)*vi
        vi_neg=(vi<0)*-vi
        
        ui_pos_norm=np.linalg.norm(ui_pos,2)
        ui_neg_norm=np.linalg.norm(ui_neg,2)
        vi_pos_norm=np.linalg.norm(vi_pos,2)
        vi_neg_norm=np.linalg.norm(vi_neg,2)
        
        norm_pos=ui_pos_norm*vi_pos_norm
        norm_neg=ui_neg_norm*vi_neg_norm
        
        if norm_pos>=norm_neg:
            w[:,i]=np.sqrt(s[i]*norm_pos)/ui_pos_norm*ui_pos
            h[i,:]=np.sqrt(s[i]*norm_pos)/vi_pos_norm*vi_pos.T
        else:
            w[:,i]=np.sqrt(s[i]*norm_neg)/ui_neg_norm*ui_neg
            h[i,:]=np.sqrt(s[i]*norm_neg)/vi_neg_norm*vi_neg.T

    return w,h.T


# This does not enforce non-negativity (just a direct minimization of reconstruction
# error, possibly with L2 regularization term
def matrix_factorization(ratings, U_init, V_init, latent_dims, n_iter=5000, alpha=0.0002, beta=0.02, tol=1e-3):
    '''
    D: rating matrix
    U: #users x latent_dims
    V: #items x latent_dims
    alpha: learning rate
    beta: regularization parameter'''
    D = ratings
    U = U_init
    V = V_init.T
    n_u, n_i = D.shape
    
    for step in range(n_iter):
        for i in range(n_u):
            for j in range(n_i):
                if D[i,j] > 0:
                    # calculate error for prediction
                    e_ij = D[i,j] - np.dot(U[i,:],V[:,j])

                    for k in range(latent_dims):
                        # update with gradient information
                        U[i,k] = U[i,k] + alpha * (2 * e_ij * V[k,j] - beta * U[i,k])
                        V[k,j] = V[k,j] + alpha * (2 * e_ij * U[i,k] - beta * V[k,j])

        e = (D - U@V)[np.where(D>0)].reshape(-1)
        if(np.linalg.norm(e)<tol):
            break
    print("Number of steps: {}".format(step+1))
    return U, V.T


#Using multiplicative updates. Enforces non-negativity (e.g., Aggarwals book)
#This is a very "banana"-model - adding regularization should improve slightly.
#Mostly interested in trying this with hybrid recommender systems,  where
# initial sparse data is "densified" with content based method and only then
# using NMF/MF for final recommendations (or svd-low-rank, collaborative filtering, etc.)
def nn_matrix_factorization(ratings, U_init, V_init, latent_dims, n_iter=1000, tol=1e-3):
    D = ratings
    U = U_init
    V = V_init
    n_u, n_i = D.shape
    K = latent_dims
    err = []
    for step in range(n_iter):
        #update V
        DtU = D.T@U
        VUtU = V@U.T@U
        for i in range(n_i):
            for j in range(K):
                V[i,j] *= DtU[i,j] / VUtU[i,j]
        
        #update U
        DV = D@V
        UVtV = U@V.T@V
        for i in range(n_u):
            for j in range(K):
                U[i,j] *= DV[i,j] / UVtV[i,j]
        e = np.linalg.norm(D-U@V.T, 'fro')
        err.append(e)
        if(e<tol):
            break
    return U,V
    
    
    

In [2]:
data = np.array([[3,1,2,2,0,2],
                [4,2,3,3,4,2],
                [4,1,3,3,2,5],
                [0,3,4,4,5,0],
                [2,5,5,0,3,3],
                [1,4,0,5,0,0]], dtype=float)

In [17]:
data = np.array([[3,1,2,2,np.nan,2],
                [4,2,3,3,4,2],
                [4,1,3,3,2,5],
                [np.nan,3,4,4,5,np.nan],
                [2,5,5,np.nan,3,3],
                [1,4,np.nan,5,np.nan,np.nan]], dtype=float)

In [3]:
means=np.nanmean(np.array([[3,1,2,2,np.nan,2],
                [4,2,3,3,4,2],
                [4,1,3,3,2,5],
                [np.nan,3,4,4,5,np.nan],
                [2,5,5,np.nan,3,3],
                [1,4,np.nan,5,np.nan,np.nan]], dtype=float), axis=1).reshape(-1,1)

In [21]:
for i in range(6):
    for j in range(6):
        if np.isnan(data[i,j]):
            data[i,j]=np.random.normal(means[i],0.5)

In [19]:
data_c = data - means

In [37]:
latents=4
u,s,v=np.linalg.svd(data, full_matrices=True)
result=u[:,:latents]@np.diag(s[:latents])@v[:latents,:]
np.round(result,2)

array([[ 2.76,  1.19,  1.37,  1.94,  0.42,  2.5 ],
       [ 3.14,  1.71,  3.46,  3.39,  3.66,  2.6 ],
       [ 4.7 ,  1.09,  3.03,  2.78,  2.01,  4.33],
       [ 0.38,  3.17,  3.68,  3.81,  5.23, -0.21],
       [ 1.96,  4.97,  5.06,  0.03,  2.96,  3.01],
       [ 1.  ,  3.95,  0.16,  5.04, -0.1 , -0.07]])

In [13]:
u,s,v=np.linalg.svd(data_c, full_matrices=True)
result=u[:,:latents]@np.diag(s[:latents])@v[:latents,:]
np.round(result + means.reshape(-1,1),2)

array([[ 2.54,  0.88,  1.63,  1.93,  0.14,  2.49],
       [ 3.07,  1.97,  3.24,  3.47,  3.89,  2.8 ],
       [ 4.52,  1.1 ,  3.26,  2.98,  1.9 ,  4.49],
       [ 0.24,  3.  ,  3.92,  3.87,  5.03, -0.2 ],
       [ 1.84,  5.01,  5.11,  0.12,  2.95,  3.12],
       [ 0.94,  4.02,  0.14,  5.1 , -0.05,  0.03]])

In [15]:
latents=5
U_init,Vt_init = random_initialization(data_c, latents)

In [33]:
U_init,Vt_init = u,v.T

In [34]:
U,V=matrix_factorization(data, U_init,Vt_init, latents, n_iter=10000, alpha=0.0002, beta=0.001, tol=1e-2)

Number of steps: 10000


In [35]:
tmp=U@V.T #+ means

In [36]:
np.round(tmp,2)

array([[3.  , 1.02, 2.04, 1.95, 3.1 , 2.  ],
       [4.  , 2.  , 2.98, 3.02, 4.  , 2.  ],
       [4.  , 1.  , 3.  , 3.01, 2.  , 5.  ],
       [3.21, 2.99, 3.99, 4.02, 5.  , 4.32],
       [2.  , 4.99, 5.  , 5.62, 3.  , 3.  ],
       [1.  , 4.01, 4.63, 4.99, 2.8 , 4.99]])

In [26]:
result = data.copy()
result[0,4] = tmp[0,4]
result[3,0] = tmp[3,0]
result[3,5] = tmp[3,5]
result[4,3] = tmp[4,3]
result[5,2] = tmp[5,2]
result[5,4] = tmp[5,4]
result[5,5] = tmp[5,5]
np.round(result,2)

array([[3.  , 1.  , 2.  , 2.  , 3.27, 2.  ],
       [4.  , 2.  , 3.  , 3.  , 4.  , 2.  ],
       [4.  , 1.  , 3.  , 3.  , 2.  , 5.  ],
       [4.54, 3.  , 4.  , 4.  , 5.  , 6.98],
       [2.  , 5.  , 5.  , 5.99, 3.  , 3.  ],
       [1.  , 4.  , 5.08, 5.  , 3.47, 6.36]])

In [27]:
result[0,4]>means[0]

array([ True])

In [28]:
result[3,0]>means[3]

array([ True])

In [29]:
result[3,5]>means[3]

array([ True])

In [30]:
result[4,3]>means[4]

array([ True])

In [31]:
result[5,2]>means[5]

array([ True])

In [32]:
result[5,4]>means[5]

array([ True])

In [33]:
result[5,5]>means[5]

array([ True])

## Collab filt

In [135]:
data

array([[3., 1., 2., 2., 0., 2.],
       [4., 2., 3., 3., 4., 2.],
       [4., 1., 3., 3., 2., 5.],
       [0., 3., 4., 4., 5., 0.],
       [2., 5., 5., 0., 3., 3.],
       [1., 4., 0., 5., 0., 0.]])

In [136]:
means

array([[2.        ],
       [3.        ],
       [3.        ],
       [4.        ],
       [3.6       ],
       [3.33333333]])

In [137]:
mask = 1 - np.asarray(data==0, dtype=int)
R_c = data - means
mask

array([[1, 1, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 0]])

In [138]:
R_c

array([[ 1.        , -1.        ,  0.        ,  0.        , -2.        ,
         0.        ],
       [ 1.        , -1.        ,  0.        ,  0.        ,  1.        ,
        -1.        ],
       [ 1.        , -2.        ,  0.        ,  0.        , -1.        ,
         2.        ],
       [-4.        , -1.        ,  0.        ,  0.        ,  1.        ,
        -4.        ],
       [-1.6       ,  1.4       ,  1.4       , -3.6       , -0.6       ,
        -0.6       ],
       [-2.33333333,  0.66666667, -3.33333333,  1.66666667, -3.33333333,
        -3.33333333]])

In [139]:
def pearson(R_c, M):
    n = R_c.shape[0]
    result = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            mask = M[i,:]*M[j,:]
            u1 = mask*R_c[i,:]
            u2 = mask*R_c[j,:]
            u1 /= np.sqrt(u1@u1)
            u2 /= np.sqrt(u2@u2)
            result[i,j] = u1@u2
    return result

## Calculate the Pearson coef (subtracting the diagonal for NN calculation)

In [140]:
C=np.round(pearson(R_c, mask),3) - np.eye(6)
C

array([[ 0.   ,  0.816,  0.707,  1.   , -0.811, -0.721],
       [ 0.816,  0.   ,  0.   ,  1.   , -0.559, -0.721],
       [ 0.707,  0.   ,  0.   ,  0.316, -0.589, -0.557],
       [ 1.   ,  1.   ,  0.316,  0.   , -0.684, -0.371],
       [-0.811, -0.559, -0.589, -0.684,  0.   ,  0.905],
       [-0.721, -0.721, -0.557, -0.371,  0.905,  0.   ]])

## Check that at least 2 neighbors, make another mask

In [141]:
nn = np.sum((C>0),axis=1, keepdims=True)
nn

array([[3],
       [2],
       [2],
       [3],
       [1],
       [1]])

In [142]:
NN_mask = (nn >= 2)
NN_mask

array([[ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False]])

## Get the indices of 2 nearest neighbors

In [143]:
indices = np.argsort(C, axis=1)[:,-2:]
indices

array([[1, 3],
       [0, 3],
       [3, 0],
       [0, 1],
       [4, 5],
       [5, 4]], dtype=int64)

## Make the NN matrix, and normalize it

In [144]:
K=np.zeros((6,6))
for i in range(6):
    K[i,indices[i,0]]=1
    K[i,indices[i,1]]=1
K /= K.sum(axis=1)
K

array([[0. , 0.5, 0. , 0.5, 0. , 0. ],
       [0.5, 0. , 0. , 0.5, 0. , 0. ],
       [0.5, 0. , 0. , 0.5, 0. , 0. ],
       [0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0.5, 0.5]])

## Calculate the final predictions 
### Note: We can also include the NN_mask

In [153]:
predictions = (1-mask)*(K@R_c + means)*NN_mask
np.round(predictions, 3)

array([[ 0. ,  0. ,  0. ,  0. ,  3. , -0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 5. ,  0. ,  0. ,  0. ,  0. ,  3.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [154]:
complete_ratings = data + (1-mask)*(K@R_c + means)
np.round(complete_ratings, 3)

array([[3.   , 1.   , 2.   , 2.   , 3.   , 2.   ],
       [4.   , 2.   , 3.   , 3.   , 4.   , 2.   ],
       [4.   , 1.   , 3.   , 3.   , 2.   , 5.   ],
       [5.   , 3.   , 4.   , 4.   , 5.   , 3.5  ],
       [2.   , 5.   , 5.   , 2.633, 3.   , 3.   ],
       [1.   , 4.   , 2.367, 5.   , 1.367, 1.367]])

In [155]:
complete_ratings = data + NN_mask*(1-mask)*(K@R_c + means)
np.round(complete_ratings, 3)

array([[3. , 1. , 2. , 2. , 3. , 2. ],
       [4. , 2. , 3. , 3. , 4. , 2. ],
       [4. , 1. , 3. , 3. , 2. , 5. ],
       [5. , 3. , 4. , 4. , 5. , 3.5],
       [2. , 5. , 5. , 0. , 3. , 3. ],
       [1. , 4. , 0. , 5. , 0. , 0. ]])